# Chatbot

In [1]:
import pandas as pd
import numpy as np

### The data:

In [2]:
data = pd.read_csv("Questions_Rep.csv", sep="\t")
data.head(10)

,Questions,Réponses
0,Que récolte le patch?,Le patch récolte plusieurs paramètres physiol...
1,Combien coûte le patch?,Il faut compter environ une cinquantaine d eu...
2,Où vont mes données?,Ils vont sur un serveur sécurisé pour permett...
3,Qui récolte mes données?,Les données sont récoltées sur un serveur séc...
4,Que fais-ce patch?,Il récolte des données physiologiques pour su...
5,Qui dirige Devinnova ?,Le directeur est Fabrice Vaussenat.
6,Qui est le directeur de Devinnova ?,Le directeur est Fabrice Vaussenat.
7,Qui puis-je appeler pour plus de renseignements ?,Voici le numéro du service communication : 061...
8,"Devinnova, c est quoi ?",Elle est une entreprise qui souhaite mettre l...
9,Pourquoi récolter mes données ?,Pour pouvoir traiter l activité de votre coeu...


In [3]:
data.shape

(15, 2)

### Cleaning:

In [4]:
# Here we pass all the characters into lowercases:

data["Questions"] = data["Questions"].str.lower()
data["Réponses"] = data["Réponses"].str.lower()
data.head()

,Questions,Réponses
0,que récolte le patch?,le patch récolte plusieurs paramètres physiol...
1,combien coûte le patch?,il faut compter environ une cinquantaine d eu...
2,où vont mes données?,ils vont sur un serveur sécurisé pour permett...
3,qui récolte mes données?,les données sont récoltées sur un serveur séc...
4,que fais-ce patch?,il récolte des données physiologiques pour su...


In [5]:
# Here we remove all the punctuation:

import re

data["Questions"] = data["Questions"].apply(lambda x: re.sub(r'(\?)|(\!)|(\.)|(\,)|(\")', '', x))
data["Réponses"] = data["Réponses"].apply(lambda x: re.sub(r'(\?)|(\!)|(\.)|(\,)|(\")|(\')', '', x))

data["Questions"] = data["Questions"].apply(lambda x: re.sub(r'(\')|(\-)|(\s\w\s)', ' ', x))
data["Réponses"] = data["Réponses"].apply(lambda x: re.sub(r'(\')|(\-)|(\s\w\s)', ' ', x))

data.head()

,Questions,Réponses
0,que récolte le patch,le patch récolte plusieurs paramètres physiol...
1,combien coûte le patch,il faut compter environ une cinquantaine euros
2,où vont mes données,ils vont sur un serveur sécurisé pour permett...
3,qui récolte mes données,les données sont récoltées sur un serveur séc...
4,que fais ce patch,il récolte des données physiologiques pour su...


In [6]:
data["Réponses"][10]

' électrocardiogramme est le tracé de activité électrique du coeur il est une représentation graphique de activité du coeur grâce ce tracé il est possible après calcul de connaître la fréquence réspiratoire par exemple  '

In [7]:
# Here we operate a tokenization : we split the sentences by words:

data["Questions"] = data["Questions"].apply(lambda x: x.split())
data["Réponses"] = data["Réponses"].apply(lambda x: x.split())
data.head()

,Questions,Réponses
0,"[que, récolte, le, patch]","[le, patch, récolte, plusieurs, paramètres, ph..."
1,"[combien, coûte, le, patch]","[il, faut, compter, environ, une, cinquantaine..."
2,"[où, vont, mes, données]","[ils, vont, sur, un, serveur, sécurisé, pour, ..."
3,"[qui, récolte, mes, données]","[les, données, sont, récoltées, sur, un, serve..."
4,"[que, fais, ce, patch]","[il, récolte, des, données, physiologiques, po..."


In [8]:
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = set(stopwords.words('french'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sarah\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
# Here we remove the stop words from our sentences:

data["Questions"] = data["Questions"].apply(lambda q: [word for word in q if word not in stop_words])
data["Réponses"] = data["Réponses"].apply(lambda r: [word for word in r if word not in stop_words])

data.head()

,Questions,Réponses
0,"[récolte, patch]","[patch, récolte, plusieurs, paramètres, physio..."
1,"[combien, coûte, patch]","[faut, compter, environ, cinquantaine, euros]"
2,"[où, vont, données]","[vont, serveur, sécurisé, permettre, médecin, ..."
3,"[récolte, données]","[données, récoltées, serveur, sécurisé, relié,..."
4,"[fais, patch]","[récolte, données, physiologiques, suivre, éta..."


In [10]:
# we have to remove the [] of the lists in each row :

data["Questions"] = data["Questions"].apply(lambda x: " ".join(x))
data["Réponses"] = data["Réponses"].apply(lambda x: " ".join(x))

data.head()

,Questions,Réponses
0,récolte patch,patch récolte plusieurs paramètres physiologiq...
1,combien coûte patch,faut compter environ cinquantaine euros
2,où vont données,vont serveur sécurisé permettre médecin avoir ...
3,récolte données,données récoltées serveur sécurisé relié appli...
4,fais patch,récolte données physiologiques suivre état car...


In [11]:
# Function to clean the new data, the inputs:

def cln_input(sentence):
    sentence = sentence.lower()
    sentence = re.sub(r'(\?)|(\!)|(\.)|(\,)|(\")', '', sentence)
    sentence = re.sub(r'(\-)|(\')|(\s\w\s)', ' ', sentence)
    res = sentence.split()
    res = [w for w in res if w not in stop_words]
    res = " ".join(res)
    return res
        
test = input("Your question: ")

resultat = cln_input(test)
resultat

Your question: Qu'elle est l'action de ce patch?


'action patch'

### Create a ML model:

In [12]:
X = data["Questions"]

y = data["Réponses"]

In [13]:
# We append the input at the end of X to compare it to the data later:
X[len(X)]=resultat
X

0                           récolte patch
1                     combien coûte patch
2                         où vont données
3                         récolte données
4                              fais patch
5                        dirige devinnova
6                     directeur devinnova
7        puis appeler plus renseignements
8                          devinnova quoi
9               pourquoi récolter données
10                               quoi ecg
11               quoi électrocardiogramme
12                     myangel vitalsigns
13                    comment dois mettre
14    comment utiliser myangel vitalsigns
15                           action patch
Name: Questions, dtype: object

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer()
X_tfidf = vec.fit_transform(X).toarray()
# if you don't put .toarray() you'll have a sparse matrix

In [15]:
X_tfidf

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.63091934, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.77584843, 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.63226102, 0.        , 0.63226102,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.44776334, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.48260314,
        0.        , 0.        , 0.        , 0.        , 0.6193118 ,
        0.        , 0.        , 0.        , 0.    

### Now let's calculate the cosinus beetween the input and the matrice of our training data:

The cosinus that will be the nearest to 1 will be the answer that corresponds the most to our input (new question).
cosinus = 0 means totally different, "independent".

In [16]:
# Now we have to separate the input from the rest of the matrix:

x_input = X_tfidf[-1]
x_input

array([0.81607818, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.57794152, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        ])

In [17]:
len(X_tfidf)

16

In [18]:
X_tfidf = X_tfidf[:-1]
len(X_tfidf)

15

In [19]:
from sklearn.metrics.pairwise import cosine_similarity

cos = cosine_similarity(x_input.reshape(1, -1), X_tfidf)

In [20]:
cos

array([[0.36463448, 0.25878103, 0.        , 0.        , 0.3340164 ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ]])

In [21]:
np.argmax(cos)

0

So here above we can see that the question already know that is the nearest to our new question is the first one (index == 0). So we'll return the ansmwer corresponding to this first question:

In [22]:
original_data = pd.read_csv("Questions_Rep.csv", sep="\t")

In [23]:
original_data["Réponses"][np.argmax(cos)]

' Le patch récolte plusieurs paramètres physiologiques comme la température du corps, la saturation en oxygène et l électrocardiogramme en 3 dérivations. '